In [1]:
from ray import rllib, tune
from ray.rllib.contrib.alpha_zero.core.alpha_zero_trainer import AlphaZeroTrainer
from ray.rllib.policy.sample_batch import DEFAULT_POLICY_ID
from ray.rllib.policy.policy_map import PolicyMap
from ray.rllib.evaluation.episode import MultiAgentEpisode
from ray.tune.registry import register_env
#from ray.rllib.contrib.alpha_zero.environments.cartpole import CartPole
from ray.rllib.contrib.alpha_zero.models.custom_torch_models import DenseModel
from ray.rllib.models.catalog import ModelCatalog
#from wrapper.jssp_wrapper import Jssp_wrapper
import gym
from wrapper.taxi_wrapper import discretetobox2 as discretetobox
#from wrapper.taxi_wrapper import TaxiTaxi
ModelCatalog.register_custom_model("dense_model", DenseModel)
#register_env("CartPoleEnv", lambda _: CartPole())
#register_env("JsspEnv", lambda _: Jssp_wrapper())
#register_env("Taxi-v3", lambda _:discretetobox())
#register_env("Taxi-v3", lambda _:TaxiTaxi())

from copy import deepcopy
import numpy as np

train_agent=True
action_dic={0: "move south",
    1: "move north",
    2: "move east",
    3: "move west",
    4: "pickup passenger",
    5: "drop off passenger",
}        


config = {
    "framework": "torch",
    "disable_env_checking":True,
    "num_workers"       : 7,
    "rollout_fragment_length": 50,
    "train_batch_size"  : 500,
    "sgd_minibatch_size": 64,
    "lr"                : 0.0001,
    "num_sgd_iter"      : 1,
    "mcts_config"       : {
        "puct_coefficient"   : 1.5,
        "num_simulations"    : 100,
        "temperature"        : 1.0,
        "dirichlet_epsilon"  : 0.20,
        "dirichlet_noise"    : 0.03,
        "argmax_tree_policy" : False,
        "add_dirichlet_noise": True,
    },
    "ranked_rewards"    : {
        "enable": True,
    },
    "model"             : {
        "custom_model": "dense_model",

    },
}

def env_creator(env_config):
    env = discretetobox(gym.make("Taxi-v3"))
    #env = gym.make('LunarLander-v2')
    return env

# use tune to register the custom environment for the ppo trainer
tune.register_env('TaxiTaxi',env_creator)



In [2]:
env=discretetobox(gym.make("Taxi-v3"))

state=env.reset()
print(state)

{'obs': array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0.

In [3]:
agent = AlphaZeroTrainer( config=config, env='TaxiTaxi')
if train_agent:
    # checkpoint_path = analysis.get_last_checkpoint() or args.checkpoint
    ## use string number to restore pre trained agent
    # nr_restore="10"
    #checkpoint_path=f'checkpoints_az/rllib_checkpoint{nr_restore}/checkpoint_{nr_restore.zfill(6)}/checkpoint-{nr_restore}'
    #agent.load_checkpoint("checkpoints_az/rllib_checkpoint1")
    #print("awd")
    #agent.restore("checkpoints_az/rllib_checkpoint1/checkpoint_000001/checkpoint-1")
    agent.load_checkpoint("published_checkpoints/az_taxi/checkpoint-34")
    print("start training")
    for _ in range(0,10):
        agent.train()
        print(f"training iteration {_} finished")
        agent.save_checkpoint(f"training_checkpoints/checkpoints_az_taxi")
    


2022-09-22 10:22:19,247	WARNING deprecation.py:46 -- DeprecationWarning: `simple_optimizer` has been deprecated. This will raise an error in the future!
2022-09-22 10:22:51,163	INFO trainable.py:159 -- Trainable.setup took 31.922 seconds. If your trainable is slow to initialize, consider setting reuse_actors=True to reduce actor creation overheads.


In [4]:
# ray 1.13.0 ezpip uninstall -y ray
#agent = AlphaZeroTrainer( config=config,env=CartPole)
agent.load_checkpoint("/Users/felix/sciebo/masterarbeit/progra/model-based_rl/published_checkpoints/az_taxi_2/checkpoint-13")


In [5]:

import time
length_list=[]
reward_list=[]
for _ in range(10):
    policy = agent.get_policy(DEFAULT_POLICY_ID)
    action_list=[]
    #env = CartPole()
    #env = Jssp_wrapper()
    env = discretetobox(gym.make("Taxi-v3"))

    obs = env.reset()
    # env2 is copy for later going evaluation
    env2=deepcopy(env)

    episode = MultiAgentEpisode(
        PolicyMap(0,0),
        lambda _, __: DEFAULT_POLICY_ID,
        lambda: None,
        lambda _: None,
        0,
    )

    episode.user_data['initial_state'] = env.get_state()

    done = False

    while not done:
        action, _, _ = policy.compute_single_action(obs, episode=episode)
        action_list.append(action)
        #print(action_dic[action])
        obs, reward, done, _ = env.step(action)
        #print(obs)
        #env.render(render_mode='human')
        #time.sleep(0.1)
        episode.length += 1

    length_list.append(episode.length)
    reward_list.append(reward)
    env.close()

2022-09-22 10:22:51,489	WARNING deprecation.py:46 -- DeprecationWarning: `MultiAgentEpisode` has been deprecated. Use `ray.rllib.evaluation.episode.Episode` instead. This will raise an error in the future!


KeyboardInterrupt: 

In [ ]:

env3=deepcopy(env2)
for action in action_list:
    env3.step(action)
    env3.render()
env3.close()

In [ ]:
import gym
from wrapper.taxi_wrapper import discretetobox
env=discretetobox(gym.make('Taxi-v3', render_mode='human'))
state=env.reset()

taxi_row, taxi_col, pass_loc, dest_idx = env.decode(env.s)
print(taxi_row)
print(taxi_col)
print(pass_loc)
print(env.locs[pass_loc])
print((taxi_row,taxi_col))
print(env.locs[pass_loc])
print(dest_idx)
env.render()
print(state['action_mask'])
"""
    - 0: move south
    - 1: move north
    - 2: move east
    - 3: move west
    - 4: pickup passenger
    - 5: drop off passenger
"""
print(True) if (taxi_row,taxi_col) == env.locs[pass_loc] else print(False)